## Dramatron

[Dramatron](https://arxiv.org/pdf/2209.14958.pdf) is a paper published by Deepmind in 2022 describing an approach for procedurally generating stories. It starts with a logline provided by the user (e.g. "A farm boy from a desert planet joins a rebellion against an evil empire") and uses that to first generate a title and characters, which are then used to generate subsequent content like locations and scene beats. Figure 1. from the paper outlines this approach in full. 


<img src="./dramatron.png" />

In this post, we'll walk through the implementation of Dramatron in Python using `promptx` - a library to help interact with generative models and embeddings.

```bash
pip install pxx
```

`promptx` lets you define the expected output of a prompt by passing in a Pydantic schema. This is used to both provide guidance for what should be generated and validation that the correct data was created.

Let's create the basic types we'll need to implement Dramatron.

In [1]:
from enum import Enum
from typing import List
from pydantic import BaseModel


class Character(BaseModel):
    name: str
    description: str


class Location(BaseModel):
    name: str
    description: str


class PlotElement(str, Enum):
    ordinary_world = "Ordinary World"
    call_to_adventure = "Call to Adventure"
    refusal_of_the_call = "Refusal of the Call"
    supernatural_aid = "Supernatural Aid"
    crossing_the_threshold = "Crossing the Threshold"
    tests_allies_enemies = "Tests, Allies, Enemies"
    belly_of_the_whale = "Belly of the Whale"
    road_of_trials = "Road of Trials"
    reward = "Reward"
    the_road_back = "The Road Back"
    meeting_with_the_goddess = "Meeting with the Goddess"
    the_return = "The Return"


class SceneBeat(BaseModel):
    location: str
    plot_element: PlotElement
    description: str


class Story(BaseModel):
    logline: str
    title: str = None
    outline: List[SceneBeat] = None
    characters: List[Character] = None
    locations: List[Location] = None

    def __init__(self, logline, **kwargs):
        super().__init__(logline=logline, **kwargs)

Next, lets define some examples, which will be used as few shot guidance for the model. I'm using the examples from the paper for consistency. 

In [2]:
star_wars = Story(
    title="Star Wars",
    logline='''
    A science - fiction fantasy about a naive but ambitious farm boy from a 
    backwater desert who discovers powers he never knew he had when he teams 
    up with a feisty princess, a mercenary space pilot and an old wizard warrior 
    to lead a ragtag rebellion against the sinister forces of the evil Galactic 
    Empire.
    ''',
    characters=[
        Character(
            name='Luke Skywalker',
            description='''
            Luke Skywalker is the hero. A naive farm boy, he will 
            discover special powers under the guidance of mentor 
            Ben Kenobi.
            ''',
        ),
        Character(
            name='Ben Kenobi',
            description='''
            Ben Kenobi is the mentor figure. A recluse Jedi warrior, 
            he will take Luke Skywalker as apprentice .
            ''',
        ),
        Character(
            name='Dartha Vader',
            description='''
            Darth Vader is the antagonist. As a commander of the 
            evil Galactic Empire, he controls space station The 
            Death Star.
            ''',
        ),
        Character(
            name='Princess Leia',
            description='''
            Princess Leia holds the plans of the Death Star. She is 
            feisty and brave. She will become Luke's friend.
            ''',
        ),
        Character(
            name='Han Solo',
            description='''
            Han Solo is a brash mercenary space pilot of the 
            Millenium Falcon and a friend of Chebacca. He will
            take Luke on his spaceship.
            ''',
        ),
        Character(
            name='Chewbacca',
            description='''
            Chewbacca is a furry and trustful monster. He is a friend 
            of Han Solo and a copilot on the Millemium Falcon.
            ''',
        ),
    ],
    locations=[
        Location(
            name='Farm',
            description='The farm is a desert planet where Luke Skywalker lives',
        ),
    ],
    outline=[
        SceneBeat(
            location='A farm on planet Tatooine',
            plot_element=PlotElement.ordinary_world,
            description='Luke Skywalker is living a normal and humble life as a farm boy on his home planet.',
        ),
        SceneBeat(
            location='Desert of Tatooine',
            plot_element=PlotElement.call_to_adventure,
            description='''
            Luke is called to his adventure by robot R2-D2 and Ben Kenobi. 
            Luke triggers R2-D2's message from Princess Leia and is intrigued 
            by her message. When R2-D2 escapes to find Ben Kenobi, Luke follows 
            and is later saved by Kenobi, who goes on to tell Luke about his Jedi 
            heritage. Kenobi suggests that he should come with him.
            '''
        ),
        SceneBeat(
            location="Ben Kenobi's farm",
            plot_element=PlotElement.refusal_of_the_call,
            description='''
            Luke refuses Kenobi, telling him that he can take Kenobi and the 
            droids as far as Mos Eisley Spaceport - but he can't possibly leave 
            his Aunt and Uncle behind for some space adventure.
            ''',
        ),
        SceneBeat(
            location='A farm on planet Tatooine',
            plot_element=PlotElement.crossing_the_threshold,
            description='''
            When Luke discovers that the stormtroopers searching for the droids 
            would track them to his farm, he rushes to warn his Aunt and Uncle, 
            only to discover them dead by the hands of the Empire. When Luke 
            returns to Kenobi, he pledges to go with him to Alderaan and learn 
            the ways of the Force like his father before him.
            ''',
        ),
        SceneBeat(
            location='On spaceship The Millenium Falcon',
            plot_element=PlotElement.tests_allies_enemies,
            description='''
            After Luke, Kenobi, and the droids hire Han Solo and Chewbacca to 
            transport them onto Alderaan, Kenobi begins Luke's training in the 
            ways of the Force. Wielding his father's lightsaber, Kenobi 
            challenges Luke. At first, he can't do it. But then Kenobi tells 
            Luke to reach out and trust his feelings. Luke succeeds.
            ''',
        ),
        SceneBeat(
            location='On spaceship The Millenium Falcon',
            plot_element=PlotElement.belly_of_the_whale,
            description='''
            The plan to defeat the Galactic Empire is to bring the Death Star 
            plans to Alderaan so that Princess Leia's father can take them to 
            the Rebellion. However, when they arrive within the system, the 
            planet is destroyed. They come across the Death Star and are pulled 
            in by a tractor beam, now trapped within the Galactic Empire.
            ''',
        ),
        SceneBeat(
            location='On spacestation The Death Star',
            plot_element=PlotElement.road_of_trials,
            description='''
            As Kenobi goes off to deactivate the tractor beam so they can escape, 
            Luke, Han, and Chewbacca discover that Princess Leia is being held on 
            the Death Star with them. They rescue her and escape to the Millennium 
            Falcon, hoping that Kenobi has successfully deactivated the tractor 
            beam. Kenobi later sacrifices himself as Luke watches Darth Vader 
            strike him down. Luke must now avenge his fallen mentor and carry on 
            his teachings.
            ''',
        ),
        SceneBeat(
            location='On spacestation The Death Star',
            plot_element=PlotElement.reward,
            description='''
            Luke has saved the princess and retrieved the Death Star plans. 
            They now have the knowledge to destroy the Galactic Empire's 
            greatest weapon once and for all.
            ''',
        ),
        SceneBeat(
            location='On spaceship The Millenium Falcon',
            plot_element=PlotElement.the_road_back,
            description='''
            Luke, Leia, Han, Chewbacca, and the droids are headed to the hidden 
            Rebellion base with the Death Star plans. They are suddenly pursued 
            by incoming TIE-Fighters, forcing Han and Luke to take action to 
            defend the ship and escape with their lives - and the plans. They 
            race to take the plans to the Rebellion and prepare for battle.
            ''',
        ),
        SceneBeat(
            location='On fighter ship X-Wing',
            plot_element=PlotElement.meeting_with_the_goddess,
            description='''
            The Rebels - along with Luke as an X-Wing pilot - take on the Death 
            Star. The Rebellion and the Galactic Empire wage war in an epic space 
            battle. Luke is the only X-Wing pilot that was able to get within the 
            trenches of the Death Star. But Darth Vader and his wingmen are in hot 
            pursuit. Just as Darth Vader is about to destroy Luke, Han returns and 
            clears the way for Luke. Luke uses the Force to guide his aiming as he 
            fires upon the sole weak point of the deadly Death Star, destroying it 
            for good.
            ''',
        ),
        SceneBeat(
            location='At the Rebellion base',
            plot_element=PlotElement.the_return,
            description='''
            Luke and Han return to the Rebellion base, triumphant, as they receive 
            medals for the heroic journey. There is peace throughout the galaxy - at 
            least for now.
            ''',
        ),
    ],
) 
    

Now we're ready to start generating some content, but first we need to specify which model we're going to use. To use OpenAI define the following environment variables:

```bash
export PXX_DEFAULT_LLM=chatgpt
export PXX_OPENAI_API_KEY=...
export PXX_OPENAI_ORG_ID=...
```

Dramatron's first generation step is creating a title and character list based on a user defined logline. Prompt instructions are all from the paper.

In [3]:
from promptx import prompt

def write_title(logline: str) -> str:
    return prompt(
        'Suggest a alternative, original and descriptive title for a known story.',
        logline,
        examples = [
            (
                star_wars.logline,
                "The Death Star's Menace"
            ),
            (
                "Residents of San Fernando Valley are under attack by flying saucers from outer space. The aliens are extraterrestrials who seek to stop humanity from creating a doomsday weapon that could destroy the universe and unleash the living dead to stalk humans who wander into the cemetery looking for evidence of the UFOs. The hero Jeff, an airline pilot, will face the aliens.",
                "The Day The Earth Was Saved By Outer Space."
            )
        ]
    )

story = Story(
    '''
    A computer hacker learns from mysterious rebels about the true nature of his reality and his role in the war against its controllers.
    '''
)

title = write_title(story.logline)
story.title = title
story

ModuleNotFoundError: No module named 'promptx.logging'

Next we create a list of character objects based on the logline by setting `output=[Character]`. This instructs the model to generate a JSON list of characters and `promptx` then parses the response into a list of instantiated `Character` objects.

In [ ]:
def create_characters(logline: str, n=5) -> List[Character]:
    return prompt(
        f'Create {n} characters for a story.',
        input=logline,
        output=[Character],
        examples=[
            (
                star_wars.logline,
                star_wars.characters,
            ),
        ],
    ).objects

characters = create_characters(story.logline)
story.characters = characters
story

With the logline, title, and characters generated, we can now move on to created the outline, which consists of a list of scene 'beats'.

In [ ]:
def write_outline(logline: str, characters: list[Character], n=10) -> List[SceneBeat]:
    return prompt(
        f'''
        Write a sequence of {n} scene beats for a story a hero's journey structure.
        ''',
        input=dict(logline=logline, characters=characters),
        output=[SceneBeat],
        examples=[
            (
                dict(
                    logline=star_wars.logline,
                    characters=star_wars.characters,
                ),
                star_wars.outline,
            ),
        ],
    ).objects

outline = write_outline(story.logline, story.characters)
story.outline = outline
story

Next we extract the scene 'beats' from the plot outline generated in the previous step. These are the main events that happen in the story.

Each scene beat has a location name so we can use this to extract location objects. These are stored like characters. For some reason the examples in the paper don't use locations from Star Wars. They also use the story logline and just the name of the location from the scene beat instead of the description of the scene. It's unclear why these decisions were made, but for consistency we'll do the same.

In [ ]:
def extract_locations(logline: str, outline: list[SceneBeat]) -> List[Location]:
    locations = []
    for beat in outline:
        if beat.location in [l.name for l in locations]:
            continue

        response = prompt(
            '''
            Generate a location based on the story logline and location name. 
            ''',
            input=dict(logline=logline, name=beat.location),
            output=Location,
            examples=[
                (
                    dict(
                        logline="Morgan adopts a new cat, Misterio, who sets a curse on anyone that pets them.",
                        name="The Adoption Center",
                    ),
                    Location(
                        name="The Adoption Center",
                        description='''
                        The Adoption Center is a sad place, especially for an unadopted 
                        pet. It is full of walls and walls of cages and cages. Inside of 
                        each is an abandoned animal, longing for a home. The lighting is 
                        dim, gray, buzzing fluorescent.
                        ''',
                    )
                ),
                (
                    dict(
                        logline="James finds a well in his backyard that is haunted by the ghost of Sam.",
                        name="The Well",
                    ),
                    Location(
                        name="The Well",
                        description='''
                        The well is buried under grass and hedges. It is at least 
                        twenty feet deep, if not more and it is masoned with stones. 
                        It is 150 years old at least. It stinks of stale, standing 
                        water, and has vines growing up the sides. It is narrow enough 
                        to not be able to fit down if you are a grown adult human.
                        ''',
                    )
                ),
                (
                    dict(
                        logline="Mr. Dorbenson finds a book at a garage sale that tells the story of his own life. And it ends in a murder! ",
                        name="The Garage Sale",
                    ),
                    Location(
                        name="The Garage Sale",
                        description='''
                        It is a garage packed with dusty household goods and antiques. 
                        There is a box at the back that says FREE and is full of paper 
                        back books.
                        ''',
                    )
                ),
            ],
        )

        locations.append(response)
    return locations
    

locations = extract_locations(story.logline, story.outline)
story.locations = locations
story

Finally, we can write a script for each scene beat using the characters and locations generated in the previous steps.

In [ ]:
def write_scene(story: Story, beat: SceneBeat, previous_beat: SceneBeat = None) -> str:
    try:
        location = next(filter(lambda x: x.name.lower() == beat.location.lower(), story.locations))
    except StopIteration:
        print(f'No location found for {beat.location}')
        return None
    
    return prompt(
        f'''
        Write a scene for a story based on the scene beat and location.
        ''',
        input=dict(
            plot_element=beat.plot_element,
            beat_description=beat.description,
            location=location.name,
            characters=story.characters,
            logline=story.logline,
            title=story.title,
            previous_beat=previous_beat,
        ),
    )


def write_script(story: Story, k: int = None) -> list[str]:
    previous_beat = None
    scenes = []
    for beat in story.outline[:k or len(story.outline)]:
        scene = write_scene(story, beat, previous_beat)
        previous_beat = beat
        scenes.append(scene)
    return scenes

In [ ]:
script = write_script(story, k=3)
script

Let's put all of that together in a `create_story` function that takes a single logline as input and returns a fully generated story object.

In [ ]:
def create_story(logline: str) -> Story:
    story = Story(logline=logline)
    story.title = write_title(story.logline)
    story.characters = create_characters(story.logline)
    story.outline = write_outline(story.logline, story.characters)
    story.locations = extract_locations(story.logline, story.outline)
    return story